<a href="https://colab.research.google.com/github/zscore314/pricing-insurance-risk/blob/main/Aggregate_Launch_DW_SM_Video_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The `aggregate` Package
Version 1.0.0
| Stephen J. Mildenhall
| March 1, 2023

![](https://readthedocs.org/projects/aggregate/badge/?version=latest)
![](https://img.shields.io/pypi/dm/aggregate.svg)
![](https://img.shields.io/github/stars/mynl/aggregate.svg)
![](https://img.shields.io/github/forks/mynl/aggregate.svg)
![](https://img.shields.io/github/contributors/mynl/aggregate.svg)
![](https://img.shields.io/pypi/v/aggregate.svg?label=pypi)
![](https://img.shields.io/github/commit-activity/m/mynl/aggregate)
![](https://img.shields.io/pypi/pyversions/aggregate.svg)
![](https://img.shields.io/pypi/l/aggregate.svg)
![](https://img.shields.io/reddit/subreddit-subscribers/AggregateDistribution)

## Introduction 

* What is ``aggregate``?
* Why did you build it?

### About this Video 

* Sections 1-5 are mostly mechanical, how to...
* Sections 6-9 are mostly conceptual: why do that and what do you get?
* See [Pricing Insurance Risk](https://www.pricinginsurancerisk.com) (PIR) book for theoretical details. 
* Notebook is available on [www.aggregate.capital](https://www.aggregate.capital), differs slightly from the presented version, includes some bug fixes.

### More Help

``aggregate`` has a very extensive [online help manual](https://aggregate.readthedocs.io/en/latest/), that is also available as a [standalone pdf](https://aggregate.readthedocs.io/_/downloads/en/latest/pdf/).
New users should start with the [10 minute guide to aggregate](https://aggregate.readthedocs.io/en/latest/2_user_guides/2_x_10mins.html) in conjunction with the [student guide](https://aggregate.readthedocs.io/en/latest/2_user_guides/2_x_student.html) for actuarial science majors or the [actuary guide](https://aggregate.readthedocs.io/en/latest/2_user_guides/2_x_actuary_student.html) for more experienced actuarial students and qualified actuaries.  

See links throughout to the relevant help sections. 

## Running Order

### Part I: For Reinsurance and Large Account Pricing Actuaries 

1. Install ``aggregate`` and upgrade ``matplotlib``
1. import 
1. Insurance example: commercial auto risk retention group 
1. Apply reinsurance
1. Pricing with Distortions

### Part II: Strategic Planning and Financial Risk Management with aggregate

6. Portfolios
1. Natural allocation pricing
1. Risk visualization 
1. Bounds, pricing envelopes and other topics 


## ``aggregate`` for R Users

Use ``aggregate`` from R via the [``reticulate``](https://rstudio.github.io/reticulate/) package.  /EOM


# New Section

## 1. Install

One-time installation.

In [ ]:
try: 
    import aggregate
except ModuleNotFoundError:
    !pip install aggregate 
    # force reload
    import os
    os.kill(os.getpid(), 9)

# Part I: For Reinsurance and Large Account Pricing Actuaries 

## 2. Setup Jupyter and import aggregate components 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings 
warnings.simplefilter("ignore")
%config InlineBackend.figure_format = "svg"
# %load_ext autotime
%load_ext autoreload
%autoreload 2

All ``aggregate`` functionality exposed through the ``build`` object. 

In [ ]:
import aggregate
from aggregate import build, qd 
print(f'Version {aggregate.__version__}')
# expect 0.11.*

**Help**

* [The Underwriter class intro](https://aggregate.readthedocs.io/en/latest/2_user_guides/2_x_10mins.html#the-underwriter-class)
* [The Underwriter class detail](https://aggregate.readthedocs.io/en/latest/3_reference/3_x_Underwriter.html#underwriter-class)

In [ ]:
build

In [ ]:
# Note: in the video this function was called qlist; it has been renamed. qlist 
# returns a DataFrame; qshow just shows it with no return value. 
build.qshow('Bodoff')

In [ ]:
build.qshow('Dice')

In [ ]:
build.show('Dice04')

## 3. Insurance example: commercial auto risk retention group 

Aggregate, frequency and severity, distributions are specified using the Dec Language (DecL), designed to go from Dec page to Distribution. 

The DecL program layout is for clarity: each line is a different clause. Python automatically concatenates strings between ``()``. 

In [ ]:
a = build('agg Comm.Auto '
          '10 claims '
          '10000 xs 0 '
          'sev lognorm 50 cv 4 '
          'poisson')
qd(a)

**Help.** Creating an aggregate distribution using the DecL language:

* [Quick start](https://aggregate.readthedocs.io/en/latest/2_user_guides/2_x_10mins.html#creating-an-aggregate-distribution)
* [Details](https://aggregate.readthedocs.io/en/latest/2_user_guides/DecL/010_Aggregate.html#specifying-a-realistic-aggregate-distribution)
* [Diagnostics](https://aggregate.readthedocs.io/en/latest/2_user_guides/2_x_10mins.html#aggregate-quick-diagnostics)

* [DecL keywords](https://aggregate.readthedocs.io/en/latest/4_dec_Language_Reference.html#dec-language-grammar-specification)
* [DecL grammar](https://aggregate.readthedocs.io/en/latest/4_dec_Language_Reference.html#dec-language-grammar-specification)
* [Picture (railroad diagram) of DecL](https://aggregate.readthedocs.io/en/latest/_static/diagram.xhtml)



### Plotting

[Help](https://aggregate.readthedocs.io/en/latest/2_user_guides/2_x_10mins.html#the-plot-method)

In [ ]:
a.plot()

### Statistics and probability functions

Aggregate objects like ``a`` should act like a ``scipy.stats`` probability distribution.

[Help](https://aggregate.readthedocs.io/en/latest/2_user_guides/2_x_10mins.html#statistical-functions)

In [ ]:
a.agg_m, a.agg_sd, a.agg_cv

In [ ]:
a.est_m, a.est_sd, a.est_cv

In [ ]:
a.pdf(2000), a.cdf(2000), a.sf(2000), a.sf(10000)

In [ ]:
a.q(.99), a.q(0.999)

### How does ``aggregate`` work?

Aggregates have complicated density and distribution functions but simple characteristic functions. ``aggregate`` uses [Fast Fourier transforms](https://en.wikipedia.org/wiki/Fast_Fourier_transform) to invert the characteristic function of the aggregate distribution $A$. 

The characteristic function of a compound Poisson with expected claim count $\lambda$ is given by
$$
M_A(t) := \mathsf E[e^{it X}] = \exp(\lambda(M_X(t) - 1))
$$
where $M_X(t)$ is the characteristic function of severity $X$ computed by taking the FFT of a discretized severity. See a [raw Python implementation](https://aggregate.readthedocs.io/en/latest/2_user_guides/2_x_10mins.html#aggregate-algorithm-in-detail).

The characteristic function is a complex number version of the moment generating function $\mathsf E[e^{tX}]$. See my [CAS Forum paper](https://www.mynl.com/old/wp/ic.pdf) for more details. 

### Three alternative ways to specify exposure

As well as entering the claim count you can:

1. Enter expected loss directly
2. Enter premium and an expected loss ratio
3. Enter exposures and a unit rate

[Help](https://aggregate.readthedocs.io/en/latest/2_user_guides/DecL/020_exposure.html#the-exposure-clause)

The next example enters the expected loss. It also uses the recommended formatting, for clarity. 

In [ ]:
a2 = build('agg CommAuto2 '
           '500 loss '
           '10000 xs 0 '
           'sev lognorm 50 cv 4 '
           'poisson')
qd(a2)

In [ ]:
a3 = build('agg CommAuto3 '
           '1000 premium at 50% lr '
           '10000 xs 0 '
           'sev lognorm 50 cv 4 '
           'poisson')
qd(a3)

The next example enters exposures (e.g., number of power units, for a trucking account) and a per unit rate (in thousands). It also adds two optional arguments,``log2`` (default 16) and ``bs``. These control the number of buckets (``2**log2``) and size of each bucket used in discretization. 

In [ ]:
a4 = build('agg CommAuto4 '
           '167 exposure at 3 rate '
           '10000 xs 0 '
           'sev lognorm 50 cv 4 '
           'poisson'
           , bs=1/8, log2=17)
qd(a4)

### Error analysis

The `aggregate_error_analysis` method provides help selecting `bs` and `log2`. 

``m`` refers to expected losses. ``agg`` is computed analytically, ``est` is the ``aggregate approximation``. ``abs`` gives the absolute error and ``rel`` the relative error. 

[Help](https://aggregate.readthedocs.io/en/latest/2_user_guides/2_x_10mins.html#estimating-bucket-size-for-discretization)

In [ ]:
ff = lambda x: f'{x:.1%}' if abs(x) < 1 else f'{x:,.1f}'
qd(a4.aggregate_error_analysis(17), sparsify=False, col_space=9, float_format=ff)

## 4. Apply reinsurance 

Occurrence reinsurance can be applied by adding a clause, after severity before frequency:

* ``occurrence net of [limit] xs ]attach]`` 
* ``occurrence net of [pct] so [limit] xs [attach]``, where ``so`` stands for "share of"
* ``occurrence ceded to [limit] xs ]attach]`` 
* ...

[Introduction to reinsurance clauses](https://aggregate.readthedocs.io/en/latest/2_user_guides/2_x_10mins.html#reinsurance)

[Detailed help](https://aggregate.readthedocs.io/en/latest/2_user_guides/DecL/080_reinsurance.html#the-reinsurance-clauses)

In [ ]:
# Apply occ reins to lower risk limit 
a_net_occ = build('agg CommAuto:NetOcc '
           '10 claims '
           '10000 xs 0 '
           'sev lognorm 50 cv 4 '
           'occurrence net of 90% so 9500 xs 500 '
        #    'occurrence net of 9500 xs 500 '
           'poisson')
qd(a_net_occ)

**Important.** ``Est E[X]`` shows the net expected loss, so we no longer expect the errors to be small. 

### Plot, accessing underlying `density_df` dataframes

The discretized distribution and other statistics are available in ``a.density_df``, a Pandas dataframe. 

[Help](https://aggregate.readthedocs.io/en/latest/2_user_guides/2_x_10mins.html#the-density-df-dataframe)

In [ ]:
a.density_df.head()

In [ ]:
ax = a.density_df.p_total.plot(xlim=[0, a.q(0.99)], label='gross')
a_net_occ.density_df.p_total.plot(ax=ax, label='net occ')
ax.legend();

You thinking fast brain is probably surprised there is no impact on small aggregate loss probabilities! The bulge after 500 appears because the net distribution now has a probability mass at 500. Run

    a_net_occ.plot()

to see the severity plot.

If the 9.5M xs 500K excess layer was 100% placed, the  net distribution could be expressed with a lower policy limit. 

In [ ]:
a.plot()

In [ ]:
a_net_occ.density_df.p_sev.plot(xlim=[0, 1000], logy=True)

In [ ]:
ax = a_net_occ.reinsurance_df.p_sev_net.plot()
a_net_occ.reinsurance_df.p_sev_ceded.plot(ax=ax)
a_net_occ.density_df.p_sev.plot(ax=ax)
ax.set(xlim=[0, 1000], yscale='log')
ax.legend()

In [ ]:
a_net_occ.plot()

In [ ]:
alt = build('agg CommAuto:NetOcc2 '
           '10 claims '
           '500 xs 0 '
           'sev lognorm 50 cv 4 '
           'poisson')
qd(alt)
qd(a_net_occ)

In [ ]:
alt.plot()

### Reinsurance impact on capital at different probability thresholds

Use the quantile function ``a.q()``. The rest is pure Python/pandas.

In [ ]:
p_values = [.1, .2, .5, .8, 0.9, .95, .99, .995, .999, .9999]
dm = pd.DataFrame({'gross': [a.q(p) for p in p_values], 
              'net_occ': [a_net_occ.q(p) for p in p_values]},
                  index=map(ff, p_values))
dm.index.name = 'p'
dm['change'] = dm.net_occ / dm.gross - 1
qd(dm, float_format=ff)

### Ceded losses

Reinsurance can output ceded or net views --- but not both.

In [ ]:
ceded =  build('agg CommAuto:CededOcc '
           '10 claims '
           '10000 xs 0 '
           'sev lognorm 50 cv 4 '
           'occurrence ceded to 90% so 9500 xs 500 '
           'poisson')
qd(ceded)

In [ ]:
ceded.plot()

In [ ]:
ceded.pprogram

In [ ]:
ceded =  build('agg CommAuto:CededOcc '
           '10 claims '
           '10000 xs 0 '
           'sev lognorm 50 cv 4 '
           'occurrence ceded to 90% so 9500 xs 500 '
           'poisson')

ax = ceded.reinsurance_df.p_sev_gross.plot()
ceded.reinsurance_df.p_sev_ceded.plot(ax=ax)
ceded.reinsurance_df.p_sev_net.plot(ax=ax)
ax.set(xlim=[0, 500 + .1 * 9500 + 50], yscale='log')
ax.legend();

In [ ]:
ceded.q(.99), ceded.q(0.999)

In [ ]:
ceded.plot()

### Layering occurrence losses

Underwriters often want to "layout out" a program. Here is a split of the 10M limit into first 250, 250 xs 250, 500 xs 500, 1 xs 1, 3 xs 2, and 5 xs 5.

[Help on tower clause](https://aggregate.readthedocs.io/en/latest/2_user_guides/DecL/080_reinsurance.html#layering-losses-in-a-tower)


In [ ]:
# layer losses
a5 = build('agg CommAuto:LL '
           '10 claims '
           '10000 xs 0 '
           'sev lognorm 50 cv 4 '
           'occurrence ceded to tower [0 250 500 1000 2000 5000 10000] '
           'poisson')
qd(a5.reinsurance_occ_layer_df, sparsify=False)

### Aggregate cover

Aggregate reinsurance clauses have the syntax as occurrence. They appear after the frequency clause. 

First, use the quantile function to determine limit and attachment at 15% excess 80% in the aggregate. 

In [ ]:
agg_attach = a_net_occ.q(0.8)
agg_detach = a_net_occ.q(0.95)
agg_limit = agg_detach - agg_attach
agg_limit, agg_attach

Apply an aggregate reinsurance clause at a 90% placement.

In [ ]:
qd(a_net_occ)
a_net_occ.pprogram

In [ ]:
a_net = build('agg CommAuto:Net '
           '10 claims '
           '10000 xs 0 '
           'sev lognorm 50 cv 4 '
           'occurrence net of 90% so 9500 xs 500 '
           'poisson '
           f'aggregate net of 90% so {agg_limit} xs {agg_attach}')
qd(a_net)

In [ ]:
a_net.reinsurance_audit_df.head().stack(0)
a_net.reinsurance_report_df

In [ ]:
# impact on capital
dm['net_agg'] = [a_net.q(p) for p in p_values]
dm['agg change'] = dm.net_agg / dm.net_occ - 1
dm['total change'] = dm.net_agg / dm.gross - 1
qd(dm, float_format=ff)

### Density functions for gross, net occ, and net from density_df dataframe

In [ ]:
ax = a.density_df['p_total'].plot(xlim=[-50, a.q(0.99)], label='Gross')
a_net_occ.density_df['p_total'].plot(ax=ax, label='Net occ')
yl = ax.get_ylim()
a_net.density_df['p_total'].plot(ax=ax, label='Net')
ax.set(ylim=yl)
ax.legend();

### Survival functions for gross, net occ, and net

Lognormal is not log concave; it has a very thick tail. 

In [ ]:
# visualize
ax = a.density_df['S'].plot(xlim=[-50, a.q(0.999)], label='Gross')
a_net_occ.density_df['S'].plot(ax=ax, label='Net occ')
a_net.density_df['S'].plot(ax=ax, label='Net')
ax.set(yscale='log', ylim=[1e-6, 1])
ax.legend();

## 5. Pricing with Distortions

[Technical introduction to distortions and spectral risk measures](https://aggregate.readthedocs.io/en/latest/5_technical_guides/5_x_distortions.html)

[Introduction to the Distortion class](https://aggregate.readthedocs.io/en/latest/2_user_guides/2_x_10mins.html#the-distortion-class)



Syntax to build a distortion using DecL:

``distortion [NAME] distortion_type parameters``

Recommended families of distortions from most tail-centric to most volatility-centric.

| Name | $g(s)$ | Parameters  | 
|:----|:-----|:----|
|Constant cost of capital (CCoC) | $g(s)=d + vs$ | $v+d=1$, $0 \le d \le 1$ discount rate cost, price increases with $d$ |
| Proportional hazard | $g(s) = s^c$ |  $0 \le c \le 1$, price increases with $c$
| Wang                |$g(s) = \Phi(\Phi^{-1}(x) + \lambda)$ | $\lambda \ge 0$, price increases with $\lambda$
| Dual | $g(s) = 1 - (1 - s)^d$ | $d \ge 1$, price increases with $d$ | 
| TVaR | $g(s) = \min(1, s / (1-p)$ | $0 \le p \le 1$, price increases with $p$; TVaR$_0(X) =\mathsf E[X]$ and TVaR$_0(X) =\max(X)$.  | 

Tail-centric distortions favor cat reinsurance and buy at the top of the tower.

Volatility-centric distortions also care about earnings volatility and non-tail events. Management is often volatility-centric. 

In [ ]:
d = build('distortion myDUAL dual 1.94363')
d.plot();

### Price gross with assets at 99 percent level

[aggregate.price method](https://aggregate.readthedocs.io/en/latest/3_reference/3_x_Distribution.html#aggregate.distributions.Aggregate.price)

Eight standard pricing outputs:

1. Expected loss
1. Premium
1. Expected margin, equal premium minus expected loss 
1. Capital (surplus), equal assets minus premium 
1. Assets, equal to 99 percentile of losses
1. Expected loss ratio
1. PQ is the premium to capital leverage
1. ROE equals expected margin over capital 

From now on, the word **expected** is understood.  

In [ ]:
pr = a.price(0.99, d)
pr

Can also price by applying the distortion to any series.
Returns bid price, expected loss, and ask price.

#### Side bar: of bid/ask spread

**Ask price**: insured comes to you and asks for a quote to pay their losses. The usual situation.

**Bid price**: you approach insured and want them to buy a policy from you. They bid on the policy.

Why might you want to "sell" a policy? Insurance is usually correlated with total loss. But in some cases it's not. Two examples: 1) if you write whole life insurance and can write an annuity on the same risk you are prefectly hedged.  2) Writing *diversifying cat* risk, such as Chile quake. You are unlikely to have two bad cat losses in the same year. When your peak peril has a loss, the premium you receive from Chile helps pay losses. Chile quake policies can be regarded as part of financing (capital). You pay over expected for financing. The same applies to reinsurance. 

[distortion.price method](https://aggregate.readthedocs.io/en/latest/3_reference/3_x_Distortion.html#aggregate.spectral.Distortion.price)

In [ ]:
d.price(a.density_df.p_total, a.q(0.99), 'both')

### Distortion pricing calculation by hand

Here's the underlying calculation by hand. The steps are 

* Extract density and survival function: ``bit = a.density_df[['loss', 'p_total', 'S']]``
* Apply the distortion: ``bit['gS'] = d.g(bit.S)``
* Compute $\int g(S(x))dx$: ``bit.loc[:a.q(0.99)-a.bs, 'gS'].sum() * a.bs``


In [ ]:
bit = a.density_df[['loss', 'p_total', 'S']]
bit['gS'] = d.g(bit.S)
print(bit.loc[:a.q(0.99)-a.bs, 'gS'].sum() * a.bs, pr.iloc[0, 1])

### Compare gross, ceded, and net pricing

All using the same distortion. This "standalone" pricing is not the [PIR](https://www.pricinginsurancerisk.com) recommended approach, but it is something people do.

In [ ]:
# put different views in a dictionary 
A = {'gross': a, 'net_occ': a_net_occ, 'net': a_net, 'ceded': ceded}
# apply distortion pricing to each at 99% capital
df = pd.concat([v.price(0.99, d) for v in A.values()])
df

# Part II: Strategic Planning and Capital Modeling with aggregate

## 6. Portfolios
Multiple lines. A classic cat-noncat example.

[Help](https://aggregate.readthedocs.io/en/latest/2_user_guides/2_x_10mins.html#the-portfolio-class)

In [ ]:
p = build('port CNC '
          'agg Cat '
              '1.8 claims 100000 xs 0 sev 1000 * pareto 1.8 - 1000 poisson '
          'agg NonCat '
              '15000 loss 1000 xs 0 sev lognorm 50 cv 4 mixed gamma .3 ' 
         )
qd(p)

### Update parameters

Fails validation because bucket size too coarse for non-cat line.

Cat line indicates not enough "space" for the aggregate answer. 

More buckets, smaller bucket size.

[Portfolio.update method](https://aggregate.readthedocs.io/en/latest/3_reference/3_x_Portfolio.html#aggregate.portfolio.Portfolio.update)

In [ ]:
p.update(log2=18, bs=1, normalize=False)
qd(p)

### Set up a pricing framework

Corporate / CEO mandates

> "We will achieve a [**15%**] return on capital over the cycle." 

> "We will hold capital sufficient for [**the 200 year event, per Solvency II**]." 

Substitute your favorite values. 


In [ ]:
# solvency II standard assets
p_reg = 0.995
a = p.q(p_reg)
a

Compute CoC implied by a nice round 20000 premium. 

Assume the company is 100% equity financed so ``roe`` and ``coc`` are interchangeable. 

In [ ]:
prem = 20000.
el = p.density_df.loc[a, 'lev_total']
margin = prem - el 
roe = coc = margin / (a - prem) 
v = 1 / (1 + coc) 
d = 1 - v
coc

### Calibrate distortions
Calibrate a range of distortion functions to achieve 20000 total gross pricing. 

`strict = 'ordered'` returns my favorite "usual suspects" distortions.

[Portfolio.calibrate_distortions](https://aggregate.readthedocs.io/en/latest/3_reference/3_x_Portfolio.html#aggregate.portfolio.Portfolio.calibrate_distortions)

[Portfolio.calibrate_distortion](https://aggregate.readthedocs.io/en/latest/3_reference/3_x_Portfolio.html#aggregate.portfolio.Portfolio.calibrate_distortion)

In [ ]:
p.calibrate_distortions(ROEs=[coc], Ps=[p_reg], strict='ordered');
# p.calibrate_distortions(COCs=[coc], Ps=[p_reg], strict='ordered');
p.distortion_df

In [ ]:
# distortions available in a dictionary 
p.dists

[Distortion.s_gs_distortion](https://aggregate.readthedocs.io/en/latest/3_reference/3_x_Distortion.html#aggregate.spectral.Distortion.s_gs_distortion)

In [ ]:
# technical issue: 
# replace ccoc with a close approximation that is continuous at 0
from aggregate import Distortion
dc = p.dists['ccoc']
s = np.array([0, 1e-14, 1])
gs = dc.g(s)
d0 = Distortion.s_gs_distortion(s, gs, 'CCoC Approx')
d0.name = 'ccoc'
dc.g(1e-20), d0.g(1e-20)
p.dists['ccoc'] = d0

### Plot the distortions

The different tail / body behavior is consistent with the table above. 

In [ ]:
[v.plot() for v in p.dists.values()];

### TVaR as a Pricing Measure  

TVaR as a pricing measure uses p=0.327, i.e. the average of the worst 67.3% of outcomes. The best 32.7% of outcomes are ignored. See the last plot.

### Underwriter Behavioral Interpretation 

Here is how an underwriter behaves when pricing using simulation. First, they simulate random numbers uniformly on the y-axis (their subjective probabilities).  Then, they apply $g^{-1}$ to get non-uniform objective probabilities on the x-axis. These are more 
bunched towards small $s$, the survival probability. Small $s$ values  simulate larger losses. Thus, the underwriter skews the distribution towards larger losses, increasing the mean. For all of these distortions, the mean of losses limited at the 99.5 percentile equals 20,000, the target premium. 

## 7. Natural allocation pricing across range of distortions 

Net uses the lifted allocation based on the gross ordering (called the lifted natural allocation). 

`analyze_distortions` applies the distortions and a range of other BS pricing methods. 

BS because they "seem reasonable", but have no theoretic basis. 

[Portfolio.analyze_distortions](https://aggregate.readthedocs.io/en/latest/3_reference/3_x_Portfolio.html#aggregate.portfolio.Portfolio.analyze_distortions)

[Portfolio.analyze_distortion](https://aggregate.readthedocs.io/en/latest/3_reference/3_x_Portfolio.html#aggregate.portfolio.Portfolio.analyze_distortion)


In [ ]:
# look at pricing across methods and distortions 
ad = p.analyze_distortions(p=p_reg)

In [ ]:
# summary exhibit for each method, here is the dual transform pricing 
ad.dual_exhibit

### Compare pricing by unit across different methods 

In [ ]:
# loss ratios and premium by method, including additive traditional (BS) methods 
pricing = pd.concat((
        ad.comp_df.xs('LR', axis=0, level=1).sort_values('Cat'),
        ad.comp_df.xs('P', axis=0, level=1)), 
    axis=1, keys=['LR', 'Prem']).drop(index=['EPD', 'VaR', 'TVaR', 'MerPer'])
# work for a nicely formatted output 
fc = lambda x: f'{x:,.0f}'
fp = lambda x: f'{x:.1%}'
fp3 = lambda x: f'{x:.3%}'
qdf = lambda x: qd(x, col_space=10, formatters={
    ('LR', 'Cat'): fp, ('LR', 'NonCat'): fp, ('LR', 'total'): fp,
    ('Prem', 'Cat'): fc, ('Prem', 'NonCat'): fc, ('Prem', 'total'): fc})
qdf(pricing)

Notice that the distortion methods span the range well and appear in the expected order.

In [ ]:
# ROE and allocated capital 
equity = pd.concat((
        ad.comp_df.xs('ROE', axis=0, level=1) ,
        ad.comp_df.xs('Q', axis=0, level=1) ), 
    axis=1, keys=['ROE', 'Q'])
qdf3 = lambda x: qd(x, col_space=10, formatters={
    ('ROE', 'Cat'): fp3, ('ROE', 'NonCat'): fp3, ('ROE', 'total'): fp3,
    ('Q', 'Cat'): fc, ('Q', 'NonCat'): fc, ('Q', 'total'): fc})
qdf3(equity.loc[pricing.index])

Classical and CCoC methods have constant cost of capital. Other distortion methods
have verying cost of allocated capital. 

In [ ]:
# leverage (P:Q) and assets capital 
levg = pd.concat((
        ad.comp_df.xs('PQ', axis=0, level=1) ,
        ad.comp_df.xs('a', axis=0, level=1) ), 
    axis=1, keys=['PQ', 'a'])
qdf4 = lambda x: qd(x, col_space=10, formatters={
    ('PQ', 'Cat'): fp, ('PQ', 'NonCat'): fp, ('PQ', 'total'): fp,
    ('a', 'Cat'): fc, ('a', 'NonCat'): fc, ('a', 'total'): fc})
qdf4(levg.loc[pricing.index])

In [ ]:
# margin and el
mel = pd.concat((
        ad.comp_df.xs('M', axis=0, level=1) ,
        ad.comp_df.xs('L', axis=0, level=1) ), 
    axis=1, keys=['M', 'L'])
qdf5 = lambda x: qd(x, col_space=10, float_format=fc)
qdf5(mel.loc[pricing.index])

In [ ]:
# occurrence reinsurance to cat line, but from 80th to 98th percentile
re_attach = p.Cat.q(0.8)
re_detach = p.Cat.q(0.98)
re_limit = re_detach - re_attach
re_limit, re_attach

In [ ]:
p_net = build('port CNC:net '
          'agg Cat '
              '1.8 claims 100000 xs 0 sev 1000 * pareto 1.8 - 1000 '
              f'occurrence net of {re_limit} xs {re_attach} poisson '
          'agg NonCat '
              '15000 loss 1000 xs 0 sev lognorm 50 cv 4 mixed gamma .3 ' 
         , update=False)
p_net.update(log2=18, bs=1, normalize=False)
qd(p_net)

Note: the difference in means is caused by the reinsurance, not an error. 

In [ ]:
# price net using the same distortions 
p_net.dists = p.dists
ad_net = p_net.analyze_distortions(p=p_reg)

In [ ]:
pricing_net = pd.concat((
        ad_net.comp_df.xs('LR', axis=0, level=1).sort_values('Cat'),
        ad_net.comp_df.xs('P', axis=0, level=1) ), 
    axis=1, keys=['LR', 'Prem']).drop(index=['EPD', 'VaR', 'TVaR', 'MerPer'])
qdf(pricing_net)

Premium is no longer constant. Loss ratios increase, premium decreases.

In [ ]:
# ROE and allocated capital 
equity_net = pd.concat((
        ad_net.comp_df.xs('ROE', axis=0, level=1) ,
        ad_net.comp_df.xs('Q', axis=0, level=1) ), 
    axis=1, keys=['ROE', 'Q'])
qdf3(equity_net.loc[pricing.index])

Net ROE varies by distortion method. Net ROE **increases** with reinsurance 
because the more stable book can be written at higher leverage, with more
expensive equity and less cheap debt. 

In [ ]:
# leverage (P:Q) and assets capital 
levg_net = pd.concat((
        ad_net.comp_df.xs('PQ', axis=0, level=1) ,
        ad_net.comp_df.xs('a', axis=0, level=1) ), 
    axis=1, keys=['PQ', 'a'])
qdf4(levg_net.loc[pricing.index])

In [ ]:
# margin and el
mel_net = pd.concat((
        ad_net.comp_df.xs('M', axis=0, level=1) ,
        ad_net.comp_df.xs('L', axis=0, level=1) ), 
    axis=1, keys=['M', 'L'])
qdf5(mel_net.loc[pricing.index])

In [ ]:
qdf((pricing - pricing_net).loc[pricing.index])

### Impact of Reinsurance on Pricing by Unit

Making Cat less risky has an impact on the premium for NonCat for two reasons. 

1. Lower the net volatility of Cat makes emphasizes the volatility risk of NonCat, which also tends to lower its loss ratio. 
1. Because of its impact on default states, it increases the value of insurance for NonCat, which increases premium and tends to lower the loss ratio. This effect is usually small for well capitalized companies. 

The more volatility-centric the distortion the lower the net value placed on the reinsurance. The Wang, dual and TVaR place around half the value on reinsurance of that ascribed by CCoC.

Both distortion and BS methods see an impact on the premium of both units. 

In [ ]:
qd((pricing - pricing_net)['Prem'], col_space=12, float_format=ff)

For the BS methods, the premium savings all comes from lower net losses and the cost of capital savings on lower assets.

For the distortion methods, these savings are moderated by the higher cost of capital (greater leverage, more equity-like capital) for the net book.

Here is a reconciliation of premium savings for BS methods as the difference in loss plus cost of captial savings. 

In [ ]:
el, el_net = ad.comp_df.iloc[0, -1], ad_net.comp_df.iloc[0, -1]
_ = pd.DataFrame(
    [p.q(p_reg), 
     p_net.q(p_reg), 
     p.q(p_reg) - p_net.q(p_reg), 
     roe * (p.q(p_reg) - p_net.q(p_reg)) / (1 + roe),
     el, 
     el_net, 
     el-el_net,
     (el - el_net) / (1 + roe), 
     roe * (p.q(p_reg) - p_net.q(p_reg)) / (1 + roe) + (el - el_net) / (1 + roe)], 
    index=['[1] Gross assets', 
           '[2] Net assets', 
           '[3] Difference in assets = [1] - [2]', 
           '[4] Discounted cost of capital difference = [3] * roe / (1 + roe)',
           '[5] Gross EL', 
           '[6] Net EL', 
           '[7] Difference in EL = [5] - [6]', 
           '[8] Discounted difference in EL = [7] / (1 + roe)',
           '[9] Premium difference = [4] + [8]'],
    columns=['Value']
)
_.index.name = 'Item'
qd(_, align='left', float_format=ff, col_space=15)

In [ ]:
p_ceded = build('port CNC:ceded '
          'agg Cat '
              '1.8 claims 100000 xs 0 sev 1000 * pareto 1.8 - 1000 '
              f'occurrence ceded to {re_limit} xs {re_attach} poisson '
         , update=False)
p_ceded.update(log2=18, bs=1, normalize=False)
qd(p_ceded)

### Implied walkaway loss ratios for reinsurance

If ceded LR > walkaway, then re is accretive (beneficial) and the model recommends buying it. It lowers the overall cost of capital.

If ceded LR < walkaway, then re is not accretive and the model recommends against buying it. 

**All BS methods haved the same premium difference - they all evaluate the reinsurance in the same way, and it is the same tail centric view taken by CCoC!**

The first table loooks at total premium savings across both lines. This is the appropriate corproate view of the reinsurance. 

In [ ]:
ceded_loss = p_ceded.est_m
lr_wa = ceded_loss / (pricing - pricing_net).loc[pricing.index, ('Prem', 'total')]
qd(lr_wa, float_format=fp)

The next table looks at the Cat unit's evaluation, only taking the difference in Cat unit premium, placed side-by-side with the corporate view. 

In [ ]:
ceded_loss = p_ceded.est_m
lr_wa_sa = ceded_loss / (pricing - pricing_net).loc[pricing.index, ('Prem', 'Cat')]
qd(pd.concat((lr_wa, lr_wa_sa), keys=['corporate', 'BU'], axis=1), float_format=fp, col_space=15)

## 8. Risk visualization

See PIR Section 15.5 or [Mildenhall Major (2022)](https://arxiv.org/pdf/2008.12427.pdf) Section 7 for a detailed explation of this monster figure. 

In [ ]:
fig, axs = plt.subplots(4, 3, figsize=(3 * 3.5, 4 * 2.45), constrained_layout=True)
p.apply_distortion(p.dists['dual'], efficient=False)
p.twelve_plot(fig, axs, p=0.999, p2=0.999)

In [ ]:
fig, axs = plt.subplots(4, 3, figsize=(3 * 3.5, 4 * 2.45), constrained_layout=True)
p_net.apply_distortion(p.dists['dual'], efficient=False)
p_net.twelve_plot(fig, axs, p=0.999, p2=0.999)

## 9. Bounds, pricing envelopes, and other topics 

See PIR Section 11.2.3 or my paper [Similar risks have similar prices](https://doi.org/10.1016/j.insmatheco.2022.04.006) for more explanation of this section. 

In [ ]:
from aggregate import Bounds 
bounds = Bounds(p)

The TVaR probability threshold to reproduce pricing (TVaR distortion parameter). 

In [ ]:
p_star = bounds.p_star('total', 20000, p.q(p_reg))
p_star

In [ ]:
bounds.tvar_cloud('total', premium=20000, a=p.q(p_reg), n_tps=128, s=128, kind='interp')

In [ ]:
fig, axs = plt.subplots(1,1, figsize=(3.5, 2.45))
bounds.weight_image(axs)

In [ ]:
# distortion envelopes for 20000 premium
fig, axs = plt.subplots(1,3, figsize=(3*3.5, 3.5), constrained_layout=True)
ax0, ax1, ax2 = axs.flat
axi = iter(axs.flat)
bounds.cloud_view(axs.flatten(), 0, alpha=1, pricing=True,
                    title=f'Premium={prem:,.1f}, a={a:,.0f}, p*={p_star:.3f}',
                    distortions=[{k: p.dists[k] for k in ['ccoc', 'tvar']},
                                {k: p.dists[k] for k in ['ph', 'wang', 'dual']}])
    

In [ ]:
from aggregate.bounds import similar_risks_graphs_sa
df = similar_risks_graphs_sa(None, bounds, p, p_net, roe, prem, p_reg)

Find the max/min net pricing, corresponding to the greatest/least benefit from reinsurance. ``t1`` is the price for net total.

In [ ]:
rmin = df.loc[df.t1 == df.t1.min()]
rmax = df.loc[df.t1 == df.t1.max()]
pd.concat((rmin, rmax))

Build distortions corresponding to the extremes, apply them, and summarize pricing by unit.

In [ ]:
from aggregate import Distortion 
w = float(rmin.weight.iloc[0])
p0, p1 = rmin.index.values[0]
if (p0, p1) == (0, 1):
    dmin = p.dists['ccoc']
else:
    dmin = Distortion('bitvar', shape=w, df=[p0, p1], display_name='min net')
print(w, p0, p1)

w = float(rmax.weight.iloc[0])
p0, p1 = rmax.index.values[0]
dmax = Distortion('bitvar', shape=w, df=[p0, p1], display_name='max net')
print(w, p0, p1)

In [ ]:
admn = p_net.analyze_distortion(dmin, p=p_reg, add_comps=False)
admx = p_net.analyze_distortion(dmax, p=p_reg, add_comps=False)
add = p_net.analyze_distortion(p.dists['dual'], p=p_reg, add_comps=False)
adph = p_net.analyze_distortion(p.dists['ph'], p=p_reg, add_comps=False)
adtvar = p_net.analyze_distortion(p.dists['tvar'], p=p_reg, add_comps=False)
df_net = pd.concat((admn.exhibit, admx.exhibit, add.exhibit, adph.exhibit, 
                    adtvar.exhibit))

In [ ]:
admn2 = p.analyze_distortion(dmin, p=p_reg, add_comps=False)
admx2 = p.analyze_distortion(dmax, p=p_reg, add_comps=False)
add2 = p.analyze_distortion(p.dists['dual'], p=p_reg, add_comps=False)
adph2 = p.analyze_distortion(p.dists['ph'], p=p_reg, add_comps=False)
adtvar2 = p.analyze_distortion(p.dists['tvar'], p=p_reg, add_comps=False)
df_gross = pd.concat((admn2.exhibit, admx2.exhibit, add2.exhibit, 
                      adph2.exhibit, adtvar2.exhibit))

In [ ]:
prem_summary = pd.concat((df_net, df_gross), keys=['net', 'gross']).xs('P', axis=0, level=2).unstack(0).sort_values(('total', 'net'))
prem_summary[('total', 'ceded')] = prem_summary[('total', 'gross')] - prem_summary[('total', 'net')] 
prem_summary[('Cat', 'diff')] = prem_summary[('Cat', 'gross')] - prem_summary[('Cat', 'net')] 
prem_summary[('NonCat', 'diff')] = prem_summary[('NonCat', 'gross')] - prem_summary[('NonCat', 'net')] 
prem_summary = prem_summary.sort_index(axis=1)
prem_summary

NonCat: you charge more when pooled with net because there is a smaller transfer in default and because the volatility is relatively more material.  
The more expensive the net, the less budget is available for reinsurance.

This table shows that the dual is close to the most volatility-centric distortion, and confirms that TVaR is essentially the most. (There are examples where TVaR is not the most vol-centric.) 

### Risk Progression 

SRMs are law invariance, they only depend on distributions. As a result, insurers are indifferent whether units contribute loss $X_i$ or loss $\mathsf E[X_i \mid X]$. This opens up a nice way to think about diversification benefit, splitting it into a pooling component, the difference between the standalone price of $X_i$ and $\mathsf E[X_i \mid X]$, and a subsequent portfolio effect in the caswe that the $\mathsf E[X_i \mid X]$ are not all comonotonic. The exhibits below quantify the components of this split. 

In [ ]:
from aggregate.extensions import risk_progression as rp

$\mathsf E[X_i \mid X]$ is stored in ``density_df`` as the columns ``exeqa_[unit name]``. 

In [ ]:
p.density_df.filter(regex='exeqa_[CNts]')

A plot of $\mathsf E[X_i \mid X]$ shows that the cat and noncat units have very different behaviors. Cat (orange) drives the tail. 

In [ ]:
ax = p.density_df.filter(regex='exeqa_[CNt]').plot(figsize=(3.5, 3.5))
b = p.q(0.99999)
ax.set(xlim=[0, b], ylim=[0,b])

ax = p_net.density_df.filter(regex='exeqa_[CNt]').plot(ax=ax, ls=':')
for i, l in enumerate(ax.lines[3:]):
    l.set(c=f'C{i}')
ax.legend();

The next block gives gross dual distortion pricing by unit (as above).

In [ ]:
add2.exhibit

The next dataframe and figure show the split of pricing by unit. See my forthcoming paper for more explanation. 

In [ ]:
ans = rp.full_monty(p, p.dists['dual'], 8)
ans.compare_df

In [ ]:
ans_n = rp.full_monty(p_net, p.dists['dual'], 8)
ans_n.compare_df